In [1]:
%load_ext autoreload
%autoreload 2

from io import StringIO

import anndata
import pandas as pd
import numpy as np
import scanpy
import scanpy as sc

import diffexpr

scanpy.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
scanpy.logging.print_versions()
scanpy.set_figure_params(frameon=False, color_map='magma_r')

outdir = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species'
# ! ls -lha $outdir

scanpy==1.4.6.dev9+ge632939 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.3 scipy==1.3.2 pandas==0.25.3 scikit-learn==0.22 statsmodels==0.10.2 python-igraph==0.7.1 louvain==0.6.1


# Define outdirs

In [2]:
outdir_gdrive = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de'
outdir_local = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de'
! mkdir $outdir_gdrive $outdir_local
outdirs = outdir_local, outdir_gdrive
!ls -lha $outdir_local

mkdir: cannot create directory ‘/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de’: File exists
mkdir: cannot create directory ‘/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de’: File exists
total 1.2G
drwxr-xr-x  2 olga czb 4.0K May 19 21:22 .
drwxrwxr-x 13 olga czb 8.0K May 22 13:37 ..
-rw-r--r--  1 olga czb 553M May 22 15:58 lemur_lung__diffexpr_compartment_group.h5ad
-rw-r--r--  1 olga czb 307M May 23 00:02 mouse_lung__diffexpr_broad_group.h5ad
-rw-r--r--  1 olga czb 307M May 24 08:02 mouse_lung_from_tabula-muris-senis__unified_compartments.h5ad


# Load into Scanpy

## Read **original** Mouse Raw object with all tissues -- not filtered for orthologous genes or anything

In [3]:
ll /home/olga/data_lg/data_sm_copy/czb-tabula-muris-senis/Data-objects

total 14779144
-rw-r--r-- 1 olga 4056529818 Dec  5 10:38 tabula-muris-senis-droplet-official-raw-obj.h5ad
-rw-r--r-- 1 olga 3960832087 May 18 15:02 tabula-muris-senis-droplet-official-raw-obj--no-duplicate-barcodes-per-seq-run.h5ad
-rw-r--r-- 1 olga 4071257993 May 18 15:02 tabula-muris-senis-droplet-official-raw-obj--seq-run-barcode-counts.h5ad
-rw-r--r-- 1 olga  679944698 Dec  5 10:48 tabula-muris-senis-droplet-processed-official-annotations-Lung.h5ad
-rw-rw-r-- 1 olga 2365256176 Dec  4 00:12 tabula-muris-senis-facs-official-raw-obj.h5ad


In [4]:
%%time

h5ad = '/home/olga/data_lg/data_sm_copy/czb-tabula-muris-senis/Data-objects/tabula-muris-senis-droplet-official-raw-obj--no-duplicate-barcodes-per-seq-run.h5ad'
mouse = scanpy.read_h5ad(h5ad)
print(mouse)
mouse_lung = mouse[mouse.obs.tissue == "Lung"]
mouse_lung

AnnData object with n_obs × n_vars = 238915 × 20138 
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'species', 'species_latin', 'channel', 'channel_tissue', 'sequencing_run', 'cell_barcode', 'n_barcodes'
    var: 'n_cells'
CPU times: user 2.02 s, sys: 3.23 s, total: 5.25 s
Wall time: 13.3 s


View of AnnData object with n_obs × n_vars = 23802 × 20138 
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'species', 'species_latin', 'channel', 'channel_tissue', 'sequencing_run', 'cell_barcode', 'n_barcodes'
    var: 'n_cells'

### Look at mouse.obs.head()

In [5]:
mouse_lung.obs.head()

,age,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,n_genes,sex,subtissue,tissue,tissue_free_annotation,species,species_latin,channel,channel_tissue,sequencing_run,cell_barcode,n_barcodes
index,,,,,,,,,,,,,,,,,,,
AAACGGGTCGCCCTTA-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAACGGGTCGCCCTTA,B cell,NA,B,droplet,18-F-50,1821.0,female,nan,Lung,Lung,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAACGGGTCGCCCTTA,1
AAAGATGAGCAGACTG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCAGACTG,non-classical monocyte,NA,Nonclassical Monocyte,droplet,18-F-50,1333.0,female,nan,Lung,Lung,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCAGACTG,1
AAAGATGAGCCGTCGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCCGTCGT,NK cell,NA,Natural Killer,droplet,18-F-50,1455.0,female,nan,Lung,Lung,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCCGTCGT,1
AAAGCAACATGGTAGG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAACATGGTAGG,alveolar macrophage,NA,Alveolar Macrophage,droplet,18-F-50,3322.0,female,nan,Lung,Lung,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAACATGGTAGG,1
AAAGCAAGTCTAGTGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAAGTCTAGTGT,B cell,NA,B,droplet,18-F-50,1120.0,female,nan,Lung,Lung,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAAGTCTAGTGT,1


## Read all species object with unified compartments

In [6]:
%%time
h5ad = f'{outdir}/concatenated__human-lung--lemur-lung--mouse-lung__10x__one2one_orthologs__unified_compartments.h5ad'
print(h5ad)
adata = scanpy.read_h5ad(h5ad)
adata

/home/olga/googledrive/TabulaMicrocebus/data/cross-species/concatenated__human-lung--lemur-lung--mouse-lung__10x__one2one_orthologs__unified_compartments.h5ad
CPU times: user 1.05 s, sys: 1.27 s, total: 2.32 s
Wall time: 1min 21s


AnnData object with n_obs × n_vars = 118089 × 11970 
    obs: 'age', 'batch', 'cell', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'channel_no_10x', 'channel_tissue', 'compartment', 'compartment_id', 'free_annotation', 'individual', 'location', 'magnetic.selection', 'method', 'mouse.id', 'nGene', 'nUMI', 'n_barcodes', 'n_counts', 'n_genes', 'orig.ident', 'original_channel', 'patient', 'percent.ribo', 'possibly_contaminated_barcode', 'preparation.site', 'region', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'subtissue', 'tissue', 'tissue_free_annotation', 'compartment_updated_from_human', 'compartment_updated_from_human__with_species', 'compartment_updated_from_lemur', 'compartment_updated_from_lemur__with_species', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'mouse_lemur__gene_name', 'mouse_lemur_to_human__homology_type', 'human_to_mouse__homology_type', 'mouse__gene_name', 'gene_ids-lemur', 'n_cells-mouse'

### Look at adata.obs.head()

In [7]:
adata.obs.head()

,age,batch,cell,cell_barcode,cell_ontology_class,cell_ontology_id,channel,channel_no_10x,channel_tissue,compartment,...,subtissue,tissue,tissue_free_annotation,compartment_updated_from_human,compartment_updated_from_human__with_species,compartment_updated_from_lemur,compartment_updated_from_lemur__with_species,narrow_group,broad_group,compartment_group
P2_1_AAACCTGAGAAACCAT-human,NaN,NaN,NaN,AAACCTGAGAAACCAT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AAATGCCAGATGAGAG-human,NaN,NaN,NaN,AAATGCCAGATGAGAG,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGATCCCT-human,NaN,NaN,NaN,AACACGTTCGATCCCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGCACTCT-human,NaN,NaN,NaN,AACACGTTCGCACTCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACCATGCAGCTCGCA-human,NaN,NaN,NaN,AACCATGCAGCTCGCA,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial


### Remove "-mouse" from combined annotations

In [8]:
mouse_obs_from_concatenated = adata.obs.query('species_batch == "mouse"')
mouse_obs_from_concatenated.index = mouse_obs_from_concatenated.index.str.split('-mouse').str[0]
mouse_obs_from_concatenated.head()

,age,batch,cell,cell_barcode,cell_ontology_class,cell_ontology_id,channel,channel_no_10x,channel_tissue,compartment,...,subtissue,tissue,tissue_free_annotation,compartment_updated_from_human,compartment_updated_from_human__with_species,compartment_updated_from_lemur,compartment_updated_from_lemur__with_species,narrow_group,broad_group,compartment_group
AAACGGGTCGCCCTTA-1-11-0-0,18m,NaN,MACA_18m_F_LUNG_50_AAACGGGTCGCCCTTA,AAACGGGTCGCCCTTA,B cell,NaN,MACA_18m_F_LUNG_50,NaN,MACA_18m_F_LUNG_50__Lung,NaN,...,NaN,Lung,Lung,lymphoid,lymphoid (Mouse),lymphoid,lymphoid (Mouse),B cell,B cell,lymphoid
AAAGATGAGCAGACTG-1-11-0-0,18m,NaN,MACA_18m_F_LUNG_50_AAAGATGAGCAGACTG,AAAGATGAGCAGACTG,non-classical monocyte,NaN,MACA_18m_F_LUNG_50,NaN,MACA_18m_F_LUNG_50__Lung,NaN,...,NaN,Lung,Lung,immune,immune (Mouse),myeloid,myeloid (Mouse),Monocyte,Monocyte,myeloid
AAAGATGAGCCGTCGT-1-11-0-0,18m,NaN,MACA_18m_F_LUNG_50_AAAGATGAGCCGTCGT,AAAGATGAGCCGTCGT,NK cell,NaN,MACA_18m_F_LUNG_50,NaN,MACA_18m_F_LUNG_50__Lung,NaN,...,NaN,Lung,Lung,immune,immune (Mouse),lymphoid,lymphoid (Mouse),Natural Killer,Natural Killer,lymphoid
AAAGCAACATGGTAGG-1-11-0-0,18m,NaN,MACA_18m_F_LUNG_50_AAAGCAACATGGTAGG,AAAGCAACATGGTAGG,alveolar macrophage,NaN,MACA_18m_F_LUNG_50,NaN,MACA_18m_F_LUNG_50__Lung,NaN,...,NaN,Lung,Lung,immune,immune (Mouse),myeloid,myeloid (Mouse),Macrophage,Macrophage,myeloid
AAAGCAAGTCTAGTGT-1-11-0-0,18m,NaN,MACA_18m_F_LUNG_50_AAAGCAAGTCTAGTGT,AAAGCAAGTCTAGTGT,B cell,NaN,MACA_18m_F_LUNG_50,NaN,MACA_18m_F_LUNG_50__Lung,NaN,...,NaN,Lung,Lung,lymphoid,lymphoid (Mouse),lymphoid,lymphoid (Mouse),B cell,B cell,lymphoid


## Add unified compartments

In [23]:
obs_with_compartments = mouse_lung.obs.join(mouse_obs_from_concatenated[['narrow_group', 'broad_group', 'compartment_group']].astype(str))
obs_with_compartments.head()

,age,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,n_genes,sex,subtissue,...,species,species_latin,channel,channel_tissue,sequencing_run,cell_barcode,n_barcodes,narrow_group,broad_group,compartment_group
index,,,,,,,,,,,,,,,,,,,,,
AAACGGGTCGCCCTTA-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAACGGGTCGCCCTTA,B cell,NA,B,droplet,18-F-50,1821.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAACGGGTCGCCCTTA,1,B cell,B cell,lymphoid
AAAGATGAGCAGACTG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCAGACTG,non-classical monocyte,NA,Nonclassical Monocyte,droplet,18-F-50,1333.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCAGACTG,1,Monocyte,Monocyte,myeloid
AAAGATGAGCCGTCGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCCGTCGT,NK cell,NA,Natural Killer,droplet,18-F-50,1455.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCCGTCGT,1,Natural Killer,Natural Killer,lymphoid
AAAGCAACATGGTAGG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAACATGGTAGG,alveolar macrophage,NA,Alveolar Macrophage,droplet,18-F-50,3322.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAACATGGTAGG,1,Macrophage,Macrophage,myeloid
AAAGCAAGTCTAGTGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAAGTCTAGTGT,B cell,NA,B,droplet,18-F-50,1120.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAAGTCTAGTGT,1,B cell,B cell,lymphoid


In [24]:
obs_with_compartments.compartment_group.value_counts()

myeloid        13778
lymphoid        4831
endothelial     2696
stromal         2305
epithelial       188
neural             4
Name: compartment_group, dtype: int64

In [33]:
obs_with_compartments.narrow_group.value_counts()

Monocyte                                  10319
Capillary                                  1743
Alveolar Fibroblast                        1499
B cell                                     1457
Macrophage                                 1217
Natural Killer                             1141
Interstitial Macrophage                    1115
CD8+ T                                      854
Neutrophil                                  543
CD4+ T                                      534
Adventitial Fibroblast                      517
Capillary Aerocyte                          512
Natural Killer T cell                       406
Vein                                        306
Myeloid Dendritic                           241
Airway Smooth Muscle and Myofibroblast      230
LY6G5B+ T                                   154
Basophil                                    128
Alveolar Epithelial Type 2                  122
Regulatory T                                121
Proliferating NK/T                      

## Create new object

In [25]:
mouse_lung_new_obs = mouse_lung.copy()
mouse_lung_new_obs.obs = obs_with_compartments
mouse_lung_new_obs

AnnData object with n_obs × n_vars = 23802 × 20138 
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'species', 'species_latin', 'channel', 'channel_tissue', 'sequencing_run', 'cell_barcode', 'n_barcodes', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'n_cells'

## Write newly created object to file

In [26]:
for d in outdirs:
    h5ad = f'{d}/mouse_lung_from_tabula-muris-senis__unified_compartments.h5ad'
    print(h5ad)
    mouse_lung_new_obs.write(h5ad)

/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de/mouse_lung_from_tabula-muris-senis__unified_compartments.h5ad


... storing 'narrow_group' as categorical
... storing 'broad_group' as categorical
... storing 'compartment_group' as categorical


/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de/mouse_lung_from_tabula-muris-senis__unified_compartments.h5ad


In [ ]:
mouse_lung_new_obs_min_cells_per_group = mouse_lung_new_obs[mouse_lung_new_obs.groupby('')]

# do differential expression


In [27]:
prefix = 'mouse_lung'

## compartment group

In [28]:
mouse_lung_new_obs.obs.head()

,age,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,n_genes,sex,subtissue,...,species,species_latin,channel,channel_tissue,sequencing_run,cell_barcode,n_barcodes,narrow_group,broad_group,compartment_group
index,,,,,,,,,,,,,,,,,,,,,
AAACGGGTCGCCCTTA-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAACGGGTCGCCCTTA,B cell,NA,B,droplet,18-F-50,1821.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAACGGGTCGCCCTTA,1,B cell,B cell,lymphoid
AAAGATGAGCAGACTG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCAGACTG,non-classical monocyte,NA,Nonclassical Monocyte,droplet,18-F-50,1333.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCAGACTG,1,Monocyte,Monocyte,myeloid
AAAGATGAGCCGTCGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGATGAGCCGTCGT,NK cell,NA,Natural Killer,droplet,18-F-50,1455.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGATGAGCCGTCGT,1,Natural Killer,Natural Killer,lymphoid
AAAGCAACATGGTAGG-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAACATGGTAGG,alveolar macrophage,NA,Alveolar Macrophage,droplet,18-F-50,3322.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAACATGGTAGG,1,Macrophage,Macrophage,myeloid
AAAGCAAGTCTAGTGT-1-11-0-0,18m,MACA_18m_F_LUNG_50_AAAGCAAGTCTAGTGT,B cell,NA,B,droplet,18-F-50,1120.0,female,nan,...,Mouse,Mus musculus,MACA_18m_F_LUNG_50,MACA_18m_F_LUNG_50__Lung,171005_A00111_0072_AH3YLGDMXX,AAAGCAAGTCTAGTGT,1,B cell,B cell,lymphoid


In [40]:
%%time

group = 'compartment_group'
sc.tl.rank_genes_groups(mouse_lung_new_obs, group)

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)


ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:02)
CPU times: user 2.42 s, sys: 408 ms, total: 2.83 s
Wall time: 2.83 s


### Write results to file

In [41]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    mouse_lung_new_obs.write(h5ad)
#     mouse_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')

In [ ]:
sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)

## Narrow group

In [42]:
%%time

group = 'narrow_group'
sc.tl.rank_genes_groups(mouse_lung_new_obs, group)

In [44]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    mouse_lung_new_obs.write(h5ad)

# mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)

## Broad group

In [43]:
%%time

group = 'broad_group'
sc.tl.rank_genes_groups(mouse_lung_new_obs, group)

for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    mouse_lung_new_obs.write(h5ad)

mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)


ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:09)
CPU times: user 9.06 s, sys: 2.02 s, total: 11.1 s
Wall time: 30.1 s


In [46]:
# mouse_lung_new_obs.uns['rank_genes_groups']

In [47]:
group

'broad_group'

In [48]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    mouse_lung_new_obs.write(h5ad)

# mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)

# Logistic regression method

In [ ]:
mouse_lung_new_obs.uns['rank_genes_groups']

### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    mouse_lung_new_obs.write(h5ad)
#     mouse_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')

## Broad group

In [ ]:
group = 'broad_group'

diffexpr.do_differential_expression(mouse_lung_new_obs, group, n_jobs=16)

### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    mouse_lung_new_obs.write(h5ad)
#     mouse_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')

## narrow group

In [ ]:
%%time

group = 'narrow_group'
do_differential_expression(mouse_lung_new_obs, group)

In [ ]:
sc.pl.rank_genes_groups(mouse_lung_new_obs)


In [ ]:
mouse_lung_new_obs.uns['rank_genes_groups']

### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    mouse_lung_new_obs.write(h5ad)
#     mouse_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')